In [2]:
import szokereso_functions as sf
import pandas as pd
import pathlib
'''
necessary imports
szokereso_functions.py is a file, currently in the same directory as this file (ie szokereso_core)
If you move them, you'll need to take care not to mess up the import, for which, I recommend:
https://stackoverflow.com/q/4383571/8565438
'''

"\nnecessary imports\nszokereso_functions.py is a file, currently in the same directory as this file (ie szokereso_core)\nIf you move them, you'll need to take care not to mess up the import, for which, I recommend:\nhttps://stackoverflow.com/q/4383571/8565438\n"

In [3]:
excelFile='/mnt/volume/jupyter/szokereso/negyedikfeladatUjraHat/vip_szotar_1.5.xlsx'

listOfColsWeWantToOutput = ['TITLE', 'URL', 'AGEGROUP', 'OWNTIME', 'SCRAPETIME', 'TEXT', 'AUTHOR',
                            'OUTLET', 'TOP1', 'TOP2', 'TOP4', 'TOP8', 'TOP12', 'TOP24', 'TOP48',
                            'clientdict0', 'clientdict1','clientdict2', 'clientdict3',
                            'clientdict4', 'clientdict5','clientdict6', 'clientdict7',
                            'clientdict8', 'clientdict9']

debugmode=True
path=str(pathlib.Path().absolute())

'''
Setting variables which will be used later.
excelFile is the path to the dictionary file, where the names are coming from
listOfColsWeWantToOutput is the name of the columns we are writing to csv file.
If debugmode is True, not output will be produced.
path needed to handle output locations correctly
'''

'\nSetting variables which will be used later.\nexcelFile is the path to the dictionary file, where the names are coming from\nlistOfColsWeWantToOutput is the name of the columns we are writing to csv file.\nIf debugmode is True, not output will be produced.\npath needed to handle output locations correctly\n'

In [ ]:
dfs = sf.prepareDirectoryTreeAndReturnDfs(excelFile,path)
#To get help with how the function above works, just call: help(sf.prepareDirectoryTreeAndReturnDfs)

clients=list(dfs.keys())
#ie the list 


while True:
    for client in clients:
        
        preMomFile,\
        postMomFileSzokereso,\
        inputPathAndWildcard,\
        postMomFileSzurt,\
        szokeresoResFilesPath,\
        szokeresoResFilesPathAndWildcard,\
        momentumraSzurtSzokeresesPath = sf.pathFinder(client,path=path)
        
        while True:
            todoFiles = sf.get_files_sorted_by_date_after_a_date(inputPathAndWildcard,[2020,7,1,0])
            todoFilesWithoutOutput = sf.getTodoFilesWithoutOutput(
                                        todoFiles=todoFiles,
                                        outputFiles=sf.getOutputFiles(szokeresoResFilesPathAndWildcard))
            if len(todoFilesWithoutOutput) == 0: break
            targetcsv=todoFilesWithoutOutput[0]
            targetdf = pd.read_csv(targetcsv)
            #dictionaries=getDictionaries(clientSearchlist=getSearchListFromTabName(tabName=client))

            dictionary = sf.dictionary_class('clientdict',10,sf.getSearchListFromTabName(tabName=client,excelFile=excelFile))
            
            #num_of_columns_for_each_dict={dictionary.name: dictionary.maxcolnum for dictionary in dictionaries}
            num_of_columns_for_each_dict={dictionary.name: dictionary.maxcolnum}
            for each in num_of_columns_for_each_dict:
                for index in range(num_of_columns_for_each_dict[each]):
                    targetdf[each + str(index)]=''
                    
            for icell, cell in enumerate(list(targetdf['TEXT'])):
                if type(cell) is not float:
                    sf_matchfinder=sf.matchfinder(cell,dictionary.searchlist)
                    if not len(sf_matchfinder) > dictionary.maxcolnum and len(sf_matchfinder)>0:
                        for i, e in enumerate(sf_matchfinder):
                            targetdf.loc[icell,dictionary.name+str(i)]=e
                                            
            filenamecore = targetcsv.split('/')[-1].split('.')[0]
            szokeresoResFilePathAndName=szokeresoResFilesPath+preMomFile+filenamecore+postMomFileSzokereso
            targetdf = targetdf[targetdf['clientdict0'].str.len()>0]

            classeddf = sf.equivalences(
                targetdf[[col for col in targetdf.columns if 'client' in col]],sf.getEquivalenceClasses(dfs[client]))

            classeddf.rename(columns={col:col+'clssd' for col in targetdf.columns},inplace=True)
            targetdf = targetdf.join(classeddf)
            
            print(momentumraSzurtSzokeresesPath+preMomFile+filenamecore+postMomFileSzurt)
            targetdf.drop(columns=[col for col in targetdf.columns if 'client' in col and 'clssd' not in col],inplace=True)
            targetdf.rename(columns={col:col.replace('clssd','') for col in targetdf.columns if 'client' in col},inplace=True)
            if not debugmode: targetdf[listOfColsWeWantToOutput].to_csv(momentumraSzurtSzokeresesPath+preMomFile+filenamecore+postMomFileSzurt,index=False)
            #targetdf[listOfColsWeWantToOutput].to_csv('probaout.csv',index=False)
         
    sf.doThisWhenNoFileIsFoundToProcess()

Directory tree has been built here: /mnt/volume/jupyter/szokereso/negyedikfeladatUjraHet/
(If you want to build it somewhere else, make sure  this function is in the directory you want the directory tree to be in.)
